In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar los datos
data = pd.read_csv("dataframe/champions.csv")

data = pd.get_dummies(data, columns=['Ronda'])
# Eliminar columnas irrelevantes
data = data.drop(['idPartido', 'Temporada', 'Evento', 'GolesLocal', 'GolesVisitante'], axis=1)
data.head()

# Codificar variables categóricas si es necesario

# Separar características y etiquetas
X = data.drop(['VictoriaLocal', 'Empate', 'VictoriaVisitante'], axis=1)
y = data[['VictoriaLocal', 'Empate', 'VictoriaVisitante']]

# Escalar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Convertir los datos a tensores
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

# Definir el modelo de redes neuronales convolucionales
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Tres clases: VictoriaLocal, Empate, VictoriaVisitante
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


Epoch 1/10


/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4869 - loss: 1.0149 - val_accuracy: 0.5886 - val_loss: 0.9138
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5960 - loss: 0.9157 - val_accuracy: 0.5943 - val_loss: 0.8981
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6001 - loss: 0.8943 - val_accuracy: 0.5771 - val_loss: 0.8991
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6653 - loss: 0.8291 - val_accuracy: 0.6171 - val_loss: 0.9160
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6281 - loss: 0.8694 - val_accuracy: 0.6286 - val_loss: 0.8792
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6168 - loss: 0.8470 - val_accuracy: 0.6057 - val_loss: 0.8783
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6343 - loss: 0.8314 - val_accuracy: 0.6229 - val_loss: 0.8710
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6555 - loss: 0.8322 - val_accuracy: 0.5943 - val_loss: 0.8733
Epo

In [16]:
# Guardar el modelo
tf.keras.models.save_model(model, 'modelos/modelo_redes_CNN.keras')


In [17]:
model = tf.keras.models.load_model("modelos/modelo_redes_CNN.keras")

In [18]:
# Cargar los datos
df = pd.read_csv("dataframe/champions_23_24.csv")

df = pd.get_dummies(df, columns=['Ronda'])
df['Ronda_Semi-finals'] = False
df['Ronda_Final'] = False

# Quedarnos solo con las filas de cuartos de final
cuartos_final = df[(df['Ronda_Quarter-finals'] == 1) & (df['Temporada'] == '2023-01-01')]

# Eliminar columnas irrelevantes
cuartos_final = cuartos_final.drop(['idPartido', 'Temporada', 'Evento', 'GolesLocal', 'GolesVisitante', 'VictoriaLocal', 'VictoriaVisitante', 'Empate'], axis=1)

# Escalar características (utilizamos transform en lugar de fit_transform)
# Ajustar el escalador a los datos de entrenamiento y luego transformar los datos de cuartos de final
X_cuartos_final = scaler.fit_transform(cuartos_final)


# Convertir los datos a tensores
X_cuartos_final = tf.convert_to_tensor(X_cuartos_final, dtype=tf.float32)

X_cuartos_final.shape



TensorShape([8, 30])

In [19]:
# Hacer predicciones
predicciones_cuartos = model.predict(X_cuartos_final)

# Determinar qué equipos avanzan a las semifinales
equipos_semifinal = []
for i in range(0, len(predicciones_cuartos), 2):
    prob_eq1_gana = predicciones_cuartos[i][0]
    prob_empate = predicciones_cuartos[i][1]
    prob_eq2_gana = predicciones_cuartos[i][2]
     
    equipo_local = cuartos_final.iloc[i]['Local']
    equipo_visitante = cuartos_final.iloc[i]['Visitante']
    
    print(f"{equipo_local} vs {equipo_visitante}:")
    print(f"Probabilidad de que {equipo_local} avance: {prob_eq1_gana:.2f}")
    print(f"Probabilidad de empate: {prob_empate:.2f}")
    print(f"Probabilidad de que {equipo_visitante} avance: {prob_eq2_gana:.2f}")
    
    if prob_eq1_gana > prob_eq2_gana:
        equipos_semifinal.append(equipo_local)
    elif prob_eq2_gana > prob_eq1_gana:
        equipos_semifinal.append(equipo_visitante)
    else:
        equipos_semifinal.append("Empate")

# Imprimir el cuadro de fases de resultados de partidos
print("Cuartos de final:")
for i in range(0, len(cuartos_final), 2):
    print(f"{cuartos_final.iloc[i]['Local']} vs {cuartos_final.iloc[i]['Visitante']}: {equipos_semifinal[i//2]} avanza")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
31 vs 3:
Probabilidad de que 31 avance: 0.42
Probabilidad de empate: 0.26
Probabilidad de que 3 avance: 0.33
24 vs 14:
Probabilidad de que 24 avance: 0.73
Probabilidad de empate: 0.17
Probabilidad de que 14 avance: 0.10
5 vs 28:
Probabilidad de que 5 avance: 0.04
Probabilidad de empate: 0.08
Probabilidad de que 28 avance: 0.88
16 vs 1:
Probabilidad de que 16 avance: 0.56
Probabilidad de empate: 0.24
Probabilidad de que 1 avance: 0.20
Cuartos de final:
31 vs 3: 31 avanza
24 vs 14: 24 avanza
5 vs 28: 28 avanza
16 vs 1: 16 avanza


In [20]:
# Hacer predicciones para las semifinales basadas en los equipos que avanzaron de los cuartos de final
# Definir enfrentamientos de las semifinales basados en los equipos que avanzaron de los cuartos de final

enfrentamientos_semifinal = [(equipos_semifinal[1], equipos_semifinal[2]), (equipos_semifinal[0], equipos_semifinal[3])]
predicciones_semifinal = []
equipos_final = []

for enfrentamiento in enfrentamientos_semifinal:
    equipo1 = enfrentamiento[0]
    equipo2 = enfrentamiento[1]
    indice_equipo1 = equipos_semifinal.index(equipo1)
    indice_equipo2 = equipos_semifinal.index(equipo2)
    
    # Suponiendo que 'predicciones_cuartos' contiene las predicciones para los cuartos de final
    prob_eq1_gana = predicciones_cuartos[indice_equipo1][0]
    prob_empate = predicciones_cuartos[indice_equipo1][1]
    prob_eq2_gana = predicciones_cuartos[indice_equipo1][2]
    
    prob_eq1_gana_oponente = predicciones_cuartos[indice_equipo2][0]
    prob_empate_oponente = predicciones_cuartos[indice_equipo2][1]
    prob_eq2_gana_oponente = predicciones_cuartos[indice_equipo2][2]
    
    # Simular el enfrentamiento
    prob_eq1_gana_semifinal = prob_eq1_gana * prob_eq2_gana_oponente
    prob_empate_semifinal = prob_empate * prob_empate_oponente
    prob_eq2_gana_semifinal = prob_eq2_gana * prob_eq1_gana_oponente

    # Normalizar las probabilidades para asegurar que sumen 1
    total_probabilidades = prob_eq1_gana_semifinal + prob_empate_semifinal + prob_eq2_gana_semifinal
    prob_eq1_gana_semifinal /= total_probabilidades
    prob_empate_semifinal /= total_probabilidades
    prob_eq2_gana_semifinal /= total_probabilidades
    
    predicciones_semifinal.append([prob_eq1_gana_semifinal, prob_empate_semifinal, prob_eq2_gana_semifinal])

# Imprimir las probabilidades y los resultados de los enfrentamientos de las semifinales
print("Semifinales:")
for i, enfrentamiento in enumerate(enfrentamientos_semifinal):
    equipo1 = enfrentamiento[0]
    equipo2 = enfrentamiento[1]
    prob_eq1_gana = predicciones_semifinal[i][0]
    prob_empate = predicciones_semifinal[i][1]
    prob_eq2_gana = predicciones_semifinal[i][2]
    
    print(f"{equipo1} vs {equipo2}:")
    print(f"Probabilidad de que {equipo1} avance: {prob_eq1_gana:.2f}")
    print(f"Probabilidad de empate: {prob_empate:.2f}")
    print(f"Probabilidad de que {equipo2} avance: {prob_eq2_gana:.2f}")
    
    # Determinar quién avanza
    if prob_eq1_gana > prob_eq2_gana:
        print(f"Resultado: {equipo1} avanza\n")
        equipos_final.append(equipo1)
    elif prob_eq2_gana > prob_eq1_gana:
        print(f"Resultado: {equipo2} avanza\n")
        equipos_final.append(equipo2)
    else:
        print("Resultado: Empate\n")

Semifinales:
24 vs 28:
Probabilidad de que 24 avance: 0.14
Probabilidad de empate: 0.15
Probabilidad de que 28 avance: 0.71
Resultado: 28 avanza

31 vs 16:
Probabilidad de que 31 avance: 0.01
Probabilidad de empate: 0.20
Probabilidad de que 16 avance: 0.78
Resultado: 16 avanza



In [21]:
# Definir los enfrentamientos de la final basados en los equipos semifinalistas
enfrentamiento_final = (equipos_final[0], equipos_final[1])

# Suponiendo que 'predicciones_semifinal' contiene las predicciones para las semifinales
indice_equipo1 = equipos_semifinal.index(enfrentamiento_final[0])
indice_equipo2 = equipos_semifinal.index(enfrentamiento_final[1])

# Suponiendo que 'predicciones_semifinal' contiene las predicciones para las semifinales
prob_eq1_gana_final = predicciones_semifinal[0][0] * predicciones_semifinal[1][2]
prob_empate_final = predicciones_semifinal[0][1] * predicciones_semifinal[1][1]
prob_eq2_gana_final = predicciones_semifinal[0][2] * predicciones_semifinal[1][0]

# Normalizar las probabilidades para asegurar que sumen 1
total_probabilidades = prob_eq1_gana_final + prob_empate_final + prob_eq2_gana_final
prob_eq1_gana_final /= total_probabilidades
prob_empate_final /= total_probabilidades
prob_eq2_gana_final /= total_probabilidades

# Imprimir las probabilidades y el resultado de la final
print("Final:")
print(f"{enfrentamiento_final[0]} vs {enfrentamiento_final[1]}:")
print(f"Probabilidad de que {enfrentamiento_final[0]} avance: {prob_eq1_gana_final:.2f}")
print(f"Probabilidad de empate: {prob_empate_final:.2f}")
print(f"Probabilidad de que {enfrentamiento_final[1]} avance: {prob_eq2_gana_final:.2f}")

# Determinar quién gana la final
if prob_eq1_gana_final > prob_eq2_gana_final:
    print(f"Resultado: {enfrentamiento_final[0]} gana la final\n")
elif prob_eq2_gana_final > prob_eq1_gana_final:
    print(f"Resultado: {enfrentamiento_final[1]} gana la final\n")
else:
    print("Resultado: Empate\n")



Final:
28 vs 16:
Probabilidad de que 28 avance: 0.73
Probabilidad de empate: 0.21
Probabilidad de que 16 avance: 0.06
Resultado: 28 gana la final

